# Tokenization

In [1]:
text = "This is a simple sentence."
text.split()

['This', 'is', 'a', 'simple', 'sentence.']

In [2]:
text.replace(".", " ").split()

['This', 'is', 'a', 'simple', 'sentence']

In [3]:
text = "This is a simple sentence, isn't it?"

In [4]:
from re import split
split(" |,|\?", text)

['This', 'is', 'a', 'simple', 'sentence', '', "isn't", 'it', '']

In [5]:
from nltk import word_tokenize

word_tokenize(text)

['This', 'is', 'a', 'simple', 'sentence', ',', 'is', "n't", 'it', '?']

# Stemming and Lemmatization

In [6]:
import nltk
# nltk.download("omw-1.4")
# nltk.download("punkt")
# nltk.download('averaged_perceptron_tagger')
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

### Stemming

In [7]:
stemmer = PorterStemmer()

In [8]:
words = ["word", "words", "walk", "walks", "walking", "happy", "happier"]
[stemmer.stem(w) for w in words]

['word', 'word', 'walk', 'walk', 'walk', 'happi', 'happier']

In [9]:
stemmer.stem("was")

'wa'

In [10]:
stemmer.stem("better")

'better'

### Lemmatization

In [11]:
lemmatizer = WordNetLemmatizer()
[lemmatizer.lemmatize(w) for w in words]

['word', 'word', 'walk', 'walk', 'walking', 'happy', 'happier']

In [12]:
lemmatizer.lemmatize("happier", pos=wordnet.ADJ)

'happy'

In [13]:
lemmatizer.lemmatize("was", pos=wordnet.VERB)

'be'

In [14]:
print(wordnet.VERB)
lemmatizer.lemmatize("was", pos="v")

v


'be'

### Spaghetti of lexical categories

In [15]:
from pandas import DataFrame

DataFrame([["JJ", "ADJ", "a"],
           ["VB", "VERB", "v"],
           ["RB", "ADVERB", "r"],
           ["NN, PRP, etc.", "NOUN", "n"]],
          columns=["nltk.pos_tag", "wordnet.postag", "string parameter"])

,nltk.pos_tag,wordnet.postag,string parameter
0,JJ,ADJ,a
1,VB,VERB,v
2,RB,ADVERB,r
3,"NN, PRP, etc.",NOUN,n


In [16]:
class Lemmatizer(WordNetLemmatizer):
    
    def __init__(self):
        self.POS_DICT = {"J": "a",
                    "V": "v",
                    "N": "n",
                    "R": "r"
                   }
        
    def __call__(self, text):
        words = nltk.word_tokenize(text)
        keys = [tag[1][0] for tag in nltk.pos_tag(words)]
        pos_tags = [self.POS_DICT[k] if k in self.POS_DICT.keys() else "n" for k in keys]
        lemmas = [self.lemmatize(word, tag) for (word, tag) in zip(words, pos_tags)]
        return lemmas

In [17]:
texts = ['A ball fell from a tree?', "Knocking on heaven's door?",
         'Open the door please', 'Eye for an eye', 'Donald Trump is closing the gates']

text = " ".join(texts)
text

"A ball fell from a tree? Knocking on heaven's door? Open the door please Eye for an eye Donald Trump is closing the gates"

In [18]:
lemmatizer = Lemmatizer()
lemmatizer(text)

['A',
 'ball',
 'fell',
 'from',
 'a',
 'tree',
 '?',
 'Knocking',
 'on',
 'heaven',
 "'s",
 'door',
 '?',
 'Open',
 'the',
 'door',
 'please',
 'Eye',
 'for',
 'an',
 'eye',
 'Donald',
 'Trump',
 'be',
 'close',
 'the',
 'gate']

# Stopwords

In [19]:
nltk.download("stopwords")
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\witen\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [20]:
stopwords.words("english")[:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [21]:
stopwords.words("hungarian")[:10]

['a',
 'ahogy',
 'ahol',
 'aki',
 'akik',
 'akkor',
 'alatt',
 'által',
 'általában',
 'amely']

# Text Vectoriztaion

In [22]:
import numpy as np
import pandas as pd

In [23]:
texts_split = [t.split() for t in texts]
texts_split

[['A', 'ball', 'fell', 'from', 'a', 'tree?'],
 ['Knocking', 'on', "heaven's", 'door?'],
 ['Open', 'the', 'door', 'please'],
 ['Eye', 'for', 'an', 'eye'],
 ['Donald', 'Trump', 'is', 'closing', 'the', 'gates']]

In [24]:
lemmatized = [lemmatizer.lemmatokenize(t) for t in texts]
lemmatized

AttributeError: 'Lemmatizer' object has no attribute 'lemmatokenize'

In [ ]:
tokens = word_tokenize(" ".join(texts).lower())
uniqs = list(set(tokens))
uniqs

In [ ]:
mat = np.zeros(shape=(len(texts), len(uniqs)))
mat.shape

In [ ]:
for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
        count = [l.lower() for l in lemmatized[i]].count(uniqs[j])
        mat[i,j] = count

pd.DataFrame(mat, columns=uniqs)

## Sklearn's CountVectorizer

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk import word_tokenize

In [ ]:
texts

In [ ]:
vectorizer = CountVectorizer(stop_words="english")
mat = vectorizer.fit_transform(texts)

pd.DataFrame(mat.toarray(), columns = vectorizer.get_feature_names_out())

In [ ]:
percent = (mat != 0).sum() / (mat.shape[0] * mat.shape[1])
f"{round(percent, 3)*100}% is not 0"

# Predicting outcome (SVM)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix

In [ ]:
df = pd.read_csv("BBC News Train.csv")
X = df["Text"]
Y = df["Category"]

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)
print(X_train.shape, x_test.shape)

In [ ]:
vectorizer = CountVectorizer(stop_words="english")

In [ ]:
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [ ]:
svm = SVC(kernel="linear").fit(X_train, Y_train)

In [ ]:
svm.score(X_train, Y_train)

In [ ]:
svm.score(X_test, Y_test)

In [ ]:
y_pred = svm.predict(X_test)
confusion_matrix(Y_test, y_pred, normalize="pred")

## With lemmatization

In [ ]:
vectorizer = CountVectorizer(stop_words="english", tokenizer=Lemmatizer())
X_train_l = vectorizer.fit_transform(X_train)
X_test_l = vectorizer.transform(X_test)

In [ ]:
svm2 = SVC(kernel="linear").fit(X_train_l, Y_train)

In [ ]:
svm2.score(X_train_l, Y_train)

In [ ]:
svm2.score(X_test_l, Y_test)

In [ ]:
y_pred = svm2.predict(X_test_l)
confusion_matrix(Y_test, y_pred, normalize="pred")

## With stemming

In [ ]:
class Stemmer(nltk.stem.porter.PorterStemmer):
    def __call__(self, text):
        tokens = word_tokenize(text)
        stemmed = [self.stem(t) for t in tokens]
        return stemmed

In [ ]:
vectorizer = CountVectorizer(stop_words="english", tokenizer=Stemmer())
X_train_s = vectorizer.fit_transform(X_train)
X_test_s = vectorizer.transform(X_test)

In [ ]:
svm3 = SVC(kernel="linear").fit(X_train_s, Y_train)
svm3.score(X_train_s, Y_train)

In [ ]:
svm3.score(X_test_s, Y_test)

In [ ]:
y_pred_s = svm3.predict(X_test_s)
confusion_matrix(Y_test, y_pred_s, normalize="pred")

# Predicting unlabeled data

In [ ]:
test = pd.read_csv("BBC News Test.csv")

In [ ]:
test

In [ ]:
test["Prediction"] = svm3.predict(vectorizer.transform(test["Text"]))

In [ ]:
test